# Integrar Bases de Datos con Servicios externos

## Simulando una conexión a Bases de Datos remotas

Traer datos de servidores remotos gracias a **db_link**. Conectarnos a servidores externos dentro de una consulta.

Para este modulo crearemos una base de datos: **db-remota** que simule la conexion remota. Y una tabla para los usuarios vip:

- id: integer
- fecha: date

Y añadamos unos cuantos datos:

    INSERT INTO public.vip(
	id, fecha)
		VALUES (50, '2019-01-30'),
		(53, '2017-05-10'),
		(99, '2016-04-05');



### Conectandonos a la base remota

Desde la base de datos *transporte* nos conectaremso a la bd que acabamos de crear. Iremos ilustrando el paso a paso, con todos los problemas que se puedan presentar.

#### Funcion db_link sin instalar

Toda esta string de conexion, podriamos encapsularla en una vista o vista materializada, para no exponer ni el usuario ni el password.

![](https://i.imgur.com/LbJ4qsT.png)

Y dara un error *function dblink(unknown, unknown) does not exist*. Creamos la extension en nuestra BD:

    CREATE EXTENSION dblink

#### Error:a column definition list is required for functions returning "record"

Si volvemos a ejecutar, saldra ese error. Tendremso que llamarlo como una tabla local con el statemente **AS**, e indicarle en que formato los vamos a recibir. 

![](https://i.imgur.com/TkGSCzw.png)

Volvemos a ejecutar

#### Error: permission denied for table vip

El *usuario_consulta* no tiene permisos para conectarse, se los damos con la opcion *grant wizard*

![](https://i.imgur.com/e8eEKLp.png)

y ahora... Funciona:

![](https://i.imgur.com/aDZV1KR.png)

Script Final:

    SELECT * FROM 
    dblink('dbname=db-remota
	  port=5432
	  host=127.0.0.1
	  user=usuario_consulta
	  password=etc1234',
	  'SELECT id, fecha FROM vip')
	  AS datos_remotos (id integer, fecha date);

### Haciendo un Join

![](https://i.imgur.com/AtMXo2w.png)

Script:

    SELECT * FROM pasajero
	JOIN
	dblink('dbname=db-remota
		port=5432
		host=127.0.0.1
		user=usuario_consulta
		password=etc1234',
		'SELECT id, fecha FROM vip')
		AS datos_remotos (id integer, fecha date)
	ON (pasajero.id = datos_remotos.id);

#### Usando el statement USING

*USING* es un valor especial de las consultas, cuando los dos campos que vamos a comparar son iguales, en este caso en ambas tablas son *id*:

![](https://i.imgur.com/90U3EVb.png)

### Encapsulando los datos sensibles en una *VIEW*

Creamos una nueva *view* con este SQL de nombre **vip_view**

![](https://i.imgur.com/TkGSCzw.png)

Reescribimos y ejecutamos:

    SELECT * FROM pasajero
    JOIN
    vip_view
    USING(id);

## Transaciones

Realizar procesos complejos seguros. Estructura Basica

        BEGIN
        [consultas]
        COMMIT | ROLLBACK

*BEGIN* le dice al motor de base de datos que vamos hacer lo siguiente en una sola transacion. Lo primero en *Pgadmin* es desactivar *autocommit*:

ANTES:

![](https://i.imgur.com/lStZV0B.png)

DESPUES:

![](https://i.imgur.com/PHGVdKd.png)

https://www.todopostgresql.com/comandos-de-transacciones-en-postgresql/




### Ejemplo 1.

Ya desactivada la opcion de AutoCommit, ejecutamos

![](https://i.imgur.com/mPEXA7w.png)

### Ejemplo 2

Si agregamos un *COMMIT* la transacion se realizara si todo esta bien, y si no lo agregamos, la transaccion estara abierta:

Supongamos lo ejecutamos sin el COMMMIT.

![](https://i.imgur.com/PjVwSIm.png)

Y si revisamos ahora mismo las tablas *estacion* y *pasajeros* no se veran reflejados los INSERTS. 
Para que se vean reflejados debemos hacer un COMMIT ya sea escribiendolo, o con el botoncito. 😊

Revisemos los ultimos 5 registros de cada tabla:

    SELECT * FROM public.tren
    ORDER BY id DESC LIMIT 5;

    SELECT * FROM public.estacion
    ORDER BY id DESC LIMIT 5;

Con esto es posible es crear muchas consultas en un mismo bloque de codigo, y que todas se apliquen solo y solo si todas cumplen con el su funcion. 

### Ejemplo 3

Hasta ahora las consultas realizadas han sido exitos, pero ¿que pasaria si alguna transacion falla?

Supongamos insertamos una estacion que ya existe:

![](https://i.imgur.com/xoNvpiA.png)

Lo que normalmente pasaria es que se insertaria el tren, y no se insertaria la estacion. Pero como es una transacion, debe estar toda completa. 

De esta forma se asegura la integridad de la informacion, en el modelo ACID. 

#### ROLLBACK

La palabra ROLLBACK no es muy comun usarla en transaciones tan explicitas como los INSERT, pero supongamos el siguiente caso de negocio, que solo querramos ingresar hasta cierta cantidad de pasajeros VIP, digamos 10. En la PL de INSERT podemos agregar una condicion que si ya hay 10 VIPs, y la condicion de INSERT iria justo antes del ROLLBACK, para que no se haga la insercion del pasajero 11. 

## Otras Extensiones de Postgres

Asi como usamos anteriormente *db_link* que venia instalada al momento de instalar Postgres pero que no estaba activa.

Hay extensiones para todo tipo: ML, math, etc etc

https://www.postgresql.org/docs/9.1/contrib.html

Importante saber la version:

    SELECT VERSION();

Instalaremos una extension llamada *fuzzystrmatch*, que contiene entre otras cosas varias funciones:

![](https://i.imgur.com/EQx2tCx.png)

- Levenstein, calculates the Levenshtein distance between two strings: calcula la cantidad de letras hay que cambiar para que dos palabras sean iguales.

Si en este momento usamos cualquiera de ellas, saldra un error:

    SELECT levenshtein('rusia', 'Prusia');

Por lo que antes debemos activar la extension:

    CREATE EXTENSION fuzzystrmatch;


Otra funcion avanzada que podemos usar es *difference*, que tan similar es su pronunciacion en ingles, entre 0 y 4, a mayor numero mayor similitud

    SELECT difference('rusia', 'Prusia');
    SELECT difference('osvaldo', 'oswaldo');

    SELECT difference('bear', 'bear');

Si nuestra aplicacion usa analis de voz, podriamos implementar estas dos funciones y no tener que reinventar la rueda. 